<a href="https://colab.research.google.com/github/easeandrelax/blood-sugar-level/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


🔹 Why Mount Google Drive?
Persistent Storage – Files in /content (Colab's default directory) get deleted after the session ends, but Google Drive keeps your files safe.

Easy File Access – Instead of re-uploading files every time, you can store them in Drive and load them whenever needed.

Handling Large Datasets – If you're working with large files (like images or CSVs), storing them in Drive prevents running out of Colab's limited storage.

Syncing with Other Devices – You can access the same files from different devices without re-downloading them.

In [ ]:
import os

image_folder = "/content/drive/MyDrive/train_images"
image_files = os.listdir(image_folder)

print(f"Total images found: {len(image_files)}")
print(image_files)  # Show first 10 images


Total images found: 3
['train_images', 'train.gsheet', 'train.csv']


In [ ]:
import glob

train_dir = "/content/drive/MyDrive/train_images"
image_count = len(glob.glob(train_dir + "/*.[jp][pn]g"))  # Counts JPG, JPEG, PNG
print("Number of images:", image_count)



Number of images: 0


In [ ]:
image_list = !ls /content/drive/MyDrive/train_images
image_count = len(image_list)
print("Number of images:", image_count)


Number of images: 1


In [ ]:
csv_path = "/content/drive/MyDrive/train_images/train.csv"
if os.path.exists(csv_path):
    print("✅ CSV file found!")
else:
    print("❌ CSV file missing. Please upload it.")


✅ CSV file found!


In [ ]:
import pandas as pd

# Get list of image files
image_files = os.listdir(image_folder)

# Create a DataFrame with filenames and random labels (modify labels accordingly)
df = pd.DataFrame({"Image Name": image_files, "Label": ["class_1"] * len(image_files)})

# Save CSV in Drive
csv_path = "/content/drive/MyDrive/train_images/train.csv"
df.to_csv(csv_path, index=False)

print(f"✅ CSV file created and saved at: {csv_path}")


✅ CSV file created and saved at: /content/drive/MyDrive/train_images/train.csv


In [ ]:
import pandas as pd

CSV_PATH = "/content/drive/MyDrive/train_images/train.csv"
df = pd.read_csv(CSV_PATH)

print("✅ CSV File Loaded Successfully!")
print(df.head())  # Display the first few rows


✅ CSV File Loaded Successfully!
     Image Name    Label
0  train_images  class_1
1  train.gsheet  class_1
2     train.csv  class_1


Step 1: Load and Preprocess Data

In [ ]:
!ls /content/drive/MyDrive/train_images



train.csv  train.gsheet  train_images


In [ ]:
!ls /content/drive/MyDrive/train_images

train.csv  train.gsheet  train_images


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define dataset paths
dataset_path = "/content/drive/MyDrive/train_images/"
csv_path = "/content/drive/MyDrive/train_images/train.csv"  # Update with actual file path

# Load CSV
df = pd.read_csv(csv_path)
df['image_path'] = dataset_path + df['Image Name']  # Full path to images

# Show dataset preview
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     Image Name    Label                                        image_path
0  train_images  class_1  /content/drive/MyDrive/train_images/train_images
1  train.gsheet  class_1  /content/drive/MyDrive/train_images/train.gsheet
2     train.csv  class_1     /content/drive/MyDrive/train_images/train.csv


 Preprocess Images for Regression Model

In [ ]:
image_path = '/content/drive/MyDrive/train_images'
IMG_SIZE = (224, 224)

# Function to load images and normalize them
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0  # Normalize
    return img_array

# Load all images into memory
X = np.array([load_and_preprocess_image(img) for img in df['image_path']])
y = np.array(df['Blood Sugar Level'])  # Target values

# Train-Test Split (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import UnidentifiedImageError  # Import for handling the specific error

# ... (rest of your code)
image_path = 'content/drive/MyDrive/train_images'

# Function to load images and normalize them, with error handling
def load_and_preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=IMG_SIZE)
        img_array = img_to_array(img) / 255.0  # Normalize
        return img_array
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {image_path}: {e}")
        return None  # or handle the error differently (e.g., skip the image)

# Load all images into memory, filtering out failed loads
X = np.array([img for img in [load_and_preprocess_image(img_path) for img_path in df['image_path']] if img is not None])
# Update target values to match the filtered images
valid_indices = [i for i, img in enumerate([load_and_preprocess_image(img_path) for img_path in df['image_path']]) if img is not None]
y = np.array(df['Blood Sugar Level'][valid_indices])  # Target values

# ... (rest of your code)

Error loading image /content/drive/MyDrive/train_images/train_images: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'
Error loading image /content/drive/MyDrive/train_images/train.gsheet: [Errno 95] Operation not supported: '/content/drive/MyDrive/train_images/train.gsheet'
Error loading image /content/drive/MyDrive/train_images/train.csv: cannot identify image file <_io.BytesIO object at 0x7f60fdfa5a80>
Error loading image /content/drive/MyDrive/train_images/train_images: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'
Error loading image /content/drive/MyDrive/train_images/train.gsheet: [Errno 95] Operation not supported: '/content/drive/MyDrive/train_images/train.gsheet'
Error loading image /content/drive/MyDrive/train_images/train.csv: cannot identify image file <_io.BytesIO object at 0x7f61a0b6d1c0>


KeyError: 'Blood Sugar Level'

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import UnidentifiedImageError

# Define image size
IMG_SIZE = (224, 224)  # Resize images to 224x224

# Get all image file paths using glob
image_dir = "/content/drive/MyDrive/train_images"
image_paths = glob.glob(image_dir + "/*.[jp][pn]g")  # Matches .jpg, .jpeg, .png

# Load labels from CSV (Ensure it contains 'image_path' and 'Blood Sugar Level')
df_labels = pd.read_csv("/content/drive/MyDrive/train_images/train.csv")

# Convert to DataFrame and merge with labels
df = pd.DataFrame({"image_path": image_paths})
df = df.merge(df_labels, on="image_path", how="inner")  # Ensure only valid images are included

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=IMG_SIZE)  # Resize image
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        return img_array
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Load images and filter out failed loads
images = [load_and_preprocess_image(img_path) for img_path in df['image_path']]
valid_data = [(img, df['Blood Sugar Level'][i]) for i, img in enumerate(images) if img is not None]

# Convert to numpy arrays
X = np.array([img for img, _ in valid_data])
y = np.array([label for _, label in valid_data])

print(f"Loaded {len(X)} valid images out of {len(image_paths)} total.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


KeyError: 'image_path'

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import UnidentifiedImageError

# Define image size
IMG_SIZE = (224, 224)

# Load the updated CSV file
csv_path = "/content/drive/MyDrive/train_images/train.csv"
df = pd.read_csv(csv_path)

# Ensure CSV has the correct columns
required_columns = {"Image_Name", "Label", "Image_Path"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"CSV must contain {required_columns} columns.")

# Function to load and preprocess images
def load_and_preprocess_image(Image_Path):
    try:
        img = load_img(Image_Path, target_size=IMG_SIZE)  # Load image
        img_array = img_to_array(img) / 255.0  # Normalize
        return img_array
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {Image_Path}: {e}")
        return None

# Load images and labels
images = [load_and_preprocess_image(img_path) for img_path in df["Image_Path"]]
valid_data = [(img, label) for img, label in zip(images, df["Label"]) if img is not None]

# Convert to numpy arrays
X = np.array([img for img, _ in valid_data])
y = np.array([label for _, label in valid_data])

print(f"Loaded {len(X)} valid images out of {len(df)} total.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression for blood sugar level
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16)

# Save trained model
model.save("/content/drive/MyDrive/blood_sugar_model.h5")

print("Model training complete and saved successfully!")


ValueError: CSV must contain {'Label', 'Image_Name', 'Image_Path'} columns.

In [ ]:
import os
print(os.path.exists('/content/drive/MyDrive/train_images/train.csv'))


True


In [ ]:
import imghdr
img_path = "/content/drive/MyDrive/train_images/train_images"
print(imghdr.what(img_path))  # Should return 'png', 'jpeg', etc.



IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'

In [ ]:
import os
from PIL import Image

image_dir = "/content/drive/MyDrive/train_images"
invalid_images = []

for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    try:
        with Image.open(file_path) as img:
            img.verify()  # Verify if the image is valid
    except Exception as e:
        invalid_images.append(file_path)

print(f"Invalid images: {len(invalid_images)}")
if invalid_images:
    print("Removing invalid images...")
    for img in invalid_images:
        os.remove(img)  # Remove corrupted images
    print("Invalid images removed.")
else:
    print("All images are valid.")


Invalid images: 3
Removing invalid images...


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'

In [ ]:
import os
from PIL import Image

def convert_images(image_dir):
    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        try:
            with Image.open(file_path) as img:
                img.convert("RGB").save(file_path, "PNG")  # Save as PNG
        except Exception as e:
            print(f"Skipping {filename}: {e}")

image_dir = "/content/drive/MyDrive/train_images"
convert_images(image_dir)
print("Image conversion complete.")


Skipping train_images: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'
Skipping train.gsheet: [Errno 95] Operation not supported: '/content/drive/MyDrive/train_images/train.gsheet'
Skipping train.csv: cannot identify image file '/content/drive/MyDrive/train_images/train.csv'
Image conversion complete.


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/train_images/train.csv")  # Load CSV file

print(int(df["Image_Path"].isnull().sum()))  # Count missing values


KeyError: 'Image_Path'

In [ ]:
import pandas as pd

# Load the CSV file
csv_path = "/content/drive/MyDrive/train_images/train.csv"
df = pd.read_csv(csv_path)

# Check for missing values in the 'Image_Path' column
missing_count = df["Image_Path"].isnull().sum()
print(f"Missing Image Paths: {int(missing_count)}")


KeyError: 'Image_Path'

In [ ]:
import os

path = "/content/drive/MyDrive/train_images/"
print("Files in directory:", os.listdir(path))


Files in directory: ['train_images', 'train.gsheet', 'train.csv']


In [ ]:
import os
from PIL import Image

image_dir = "/content/drive/MyDrive/train_images/train_images"

valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

def convert_images(image_dir):
    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        # Skip directories and non-image files
        if not os.path.isfile(file_path) or not any(filename.lower().endswith(ext) for ext in valid_extensions):
            print(f"Skipping {filename} (not an image file)")
            continue
        try:
            with Image.open(file_path) as img:
                img.convert("RGB").save(file_path, "PNG")  # Save as PNG
        except Exception as e:
            print(f"Skipping {filename}: {e}")

convert_images(image_dir)
print("Image conversion complete.")


In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image

image_dir = "/content/drive/MyDrive/train_images/train_images"
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

# List to store image data
image_data = []

# Read images and store pixel values
for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    if not any(filename.lower().endswith(ext) for ext in valid_extensions):
        print(f"Skipping {filename} (not an image file)")
        continue
    try:
        with Image.open(file_path) as img:
            img = img.resize((128, 128))  # Resize for uniform shape
            img_array = np.array(img)  # Convert image to NumPy array
            image_data.append(img_array.flatten())  # Flatten and store pixel values
    except Exception as e:
        print(f"Skipping {filename}: {e}")

# Convert list to Pandas DataFrame
df = pd.DataFrame(image_data)
print(df.shape)  # Check dimensions of DataFrame



In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile

# Fix for truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_dir = "/content/drive/MyDrive/train_images/train_images"
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

image_data = []

for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    if not any(filename.lower().endswith(ext) for ext in valid_extensions):
        print(f"Skipping {filename} (not an image file)")
        continue
    try:
        with Image.open(file_path) as img:
            img = img.resize((128, 128))  # Resize to 128x128
            img_array = np.array(img)  # Convert to NumPy array
            image_data.append(img_array.flatten())  # Flatten and store
    except Exception as e:
        print(f"Skipping {filename}: {e}")  # Ignore errors

# Convert to Pandas DataFrame
df = pd.DataFrame(image_data)
print(f"Shape of DataFrame: {df.shape}")  # Check dimensions


In [ ]:
img = img.convert("L")  # Convert to grayscale
img_array = np.array(img).flatten()  # 128x128 = 16384 instead of 49152


In [2]:
import matplotlib.pyplot as plt
import cv2

image_path = "your_image_path_here.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is not None:
    # Resize image to 512x512 for faster processing
    image = cv2.resize(image, (512, 512))

    plt.imshow(image, cmap='gray')
    plt.title("Grayscale Image Check")
    plt.axis("off")
    plt.show()
else:
    print("Error: Image not found!")


Error: Image not found!


In [ ]:
for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    print(f"Processing: {filename}")  # Debugging statement


In [ ]:
import os
print("Directory exists:", os.path.exists(image_dir))


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/train_images"))



In [ ]:
import matplotlib.pyplot as plt
plt.imshow(your_image, cmap='gray')
plt.show()
()

NameError: name 'your_image' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load an example image (modify the path based on your dataset)
image_path = "your_image_path_here.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale mode

# Check if the image is grayscale (should have only 2 dimensions)
if len(image.shape) == 2:
    print("The image is grayscale ✅")
else:
    print("The image is NOT grayscale ❌")

# Display the image
plt.imshow(image, cmap='gray')
plt.title("Grayscale Image")
plt.axis("off")
plt.show()


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import cv2

image_path = "/content/drive/MyDrive/train_images/train_images.jpg"  # Change this to the correct path

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is None:
    print("Error: Image not loaded. Check the file path and format!")
else:
    print("Image loaded successfully.")
    print("Image shape:", image.shape)


Error: Image not loaded. Check the file path and format!


In [ ]:
!ls /content/drive/MyDrive/train_images/


train.csv  train.gsheet  train_images


In [ ]:
import os

image_path = "/content/drive/MyDrive/train_images/7005be54cab1.png"
print("Does the file exist?", os.path.exists(image_path))


Does the file exist? False


In [ ]:
!ls /content/drive/MyDrive/train_images/*.png

!ls /content/drive/MyDrive/train_images/class_1/*.png
!ls /content/drive/MyDrive/train_images/class_2/*.png


ls: cannot access '/content/drive/MyDrive/train_images/*.png': No such file or directory
ls: cannot access '/content/drive/MyDrive/train_images/class_1/*.png': No such file or directory
ls: cannot access '/content/drive/MyDrive/train_images/class_2/*.png': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/train_images/


train.csv  train.gsheet  train_images


In [ ]:
image_path = "/content/drive/MyDrive/train_images/class_1/7005be54cab1.png"


In [3]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Set paths
data_dir = '/content/drive/My Drive/train_images'
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Model definition
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Save model
model.save('/content/drive/My Drive/retina_model.h5')

# Plot results
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()

plot_history(history)


Found 1847 images belonging to 1 classes.
Found 461 images belonging to 1 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 7/58 ━━━━━━━━━━━━━━━━━━━━ 6:20 7s/step - accuracy: 0.9882 - loss: 0.2327

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
OSError: image file is truncated
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/PIL/ImageFile.py", line 280, in load
    s = read(self.decodermaxblock)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/PIL/PngImagePlugin.py", line 989, in load_read
    cid, pos, length = self.png.read()
                       ^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/PIL/PngImagePlugin.py", line 173, in read
    length = i32(s)
             ^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/PIL/_binary.py", line 95, in i32be
    return unpack_from(">I", c, o)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^

struct.error: unpack_from requires a buffer of at least 4 bytes for unpacking 4 bytes at offset 0 (actual buffer size is 0)


The above exception was the direct cause of the following exce

In [4]:
import cv2
import os
import matplotlib.pyplot as plt

# Set your image path
image_path = "/content/drive/MyDrive/train_images"  # Change to actual image path

# Check if the image exists
if not os.path.exists(image_path):
    print("❌ Error: Image file not found! Check the path.")
else:
    # Load the image and resize it to 512x512 for faster processing
    image = cv2.imread(image_path)
    image = cv2.resize(image, (512, 512))

    # Convert to grayscale (optional)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Save grayscale image (optional)
    cv2.imwrite("grayscale_image.jpg", gray_image)

    # Show both images
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Resized Original Image")

    plt.subplot(1, 2, 2)
    plt.imshow(gray_image, cmap='gray')
    plt.title("Grayscale Image (Saved as 'grayscale_image.png')")

    plt.show()

print("✅ Processing Done!")

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [5]:
!unzip -o "/content/drive/MyDrive/train_images.zip" -d "/content/drive/MyDrive/train_images/"

Archive:  /content/drive/MyDrive/train_images.zip
  inflating: /content/drive/MyDrive/train_images/train_images/1ae8c165fd53.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b329a127307.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b32e1d775ea.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b3647865779.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b398c0494d1.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b4625877527.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b495ac025b7.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b862fb6f65d.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b8701231c8f.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1b8ad0afe9fb.png  
  inflating: /content/drive/MyDrive/train_images/train_images/1bb0ddfe753a.png  
  inflating: /content/drive/MyDrive/train_images/train_imag

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import numpy as np
import os

# Define dataset paths
train_dir = "/content/drive/MyDrive/train_images/train_images"
val_dir = "/content/drive/MyDrive/val_images"  # If you have a validation set

# Image Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",  # Binary classification [class_1, class_2]
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

# Load Pretrained ResNet50 Model (without top layers)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation="sigmoid")  # Sigmoid for binary classification

model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save the trained model
model.save("/content/drive/MyDrive/blood_sugar_model.h5")

# Predict on a test image
def predict_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    return "class_1" if prediction[0][0] < 0.5 else "class_2"

# Example usage
test_image_path = "/content/drive/MyDrive/test_image.png"  # Replace with actual path
print("Prediction:", predict_image(test_image_path))


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


ValueError: All `outputs` values must be KerasTensors. Received: outputs=<Dense name=dense_3, built=False> including invalid value <Dense name=dense_3, built=False> of type <class 'keras.src.layers.core.dense.Dense'>

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split

# Load dataset CSV (Assuming it's structured as: filename, blood_sugar_level)
csv_path = "/content/drive/MyDrive/train_images/train.csv"
data = pd.read_csv(csv_path)

# Path to images directory
image_dir = "/content/drive/MyDrive/train_images/"

# Function to load and preprocess images
def load_and_preprocess_image(filename):
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Load as RGB
    img = cv2.resize(img, (224, 224))  # Resize to match CNN input size
    img = img / 255.0  # Normalize pixel values (0-1)
    return img

# Load images and labels
X = np.array([load_and_preprocess_image(fname) for fname in data["filename"]])
y = np.array(data["blood_sugar_level"])  # Continuous values for regression

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="linear")  # Regression output (continuous value)
])

# Compile Model (Mean Squared Error loss for regression)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train Model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate Model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")


KeyError: 'filename'

In [11]:
import pandas as pd

csv_path = "/content/drive/MyDrive/train_images/Images_train.csv"
data = pd.read_csv(csv_path)

print(data.head())  # Check column names


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/Images_train.csv'

In [12]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Step 1: Load the CSV file
csv_path = "/content/drive/MyDrive/train_images/train.csv"
data = pd.read_csv(csv_path)

# Step 2: Convert labels (class_1 -> 0, class_2 -> 1)
data["Label"] = data["Label"].map({"class_1": 0, "class_2": 1})

# Step 3: Load images and preprocess
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)  # Read image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (128, 128))  # Resize to 128x128
    img = img / 255.0  # Normalize pixel values
    return img

# Step 4: Prepare dataset
X = np.array([load_and_preprocess_image(img_path) for img_path in data["Image_Path"]])
y = np.array(data["Label"])

# Step 5: Split dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shape of dataset
print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

# --- 1. Data Loading and Preprocessing ---
LABELS_CSV = "/content/drive/MyDrive/train_images/train.csv"

# Load CSV
labels_df = pd.read_csv(LABELS_CSV)

# Extract Image Paths and Labels
image_filenames = labels_df["Image_Path"].tolist()
targets = labels_df["Label"].values

# Convert Labels to Numeric Values
label_encoder = LabelEncoder()
targets = label_encoder.fit_transform(targets)

# Function to Load and Preprocess Images
def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert("RGB")
        img = img.resize((224, 224))  # Resize to 224x224
        img_array = np.array(img) / 255.0  # Normalize pixel values
        return img_array
    except Exception as e:
        print(f"Error loading image: {image_path} - {e}")
        return None

# Load Images
images = []
valid_targets = []

for filename, target in zip(image_filenames, targets):
    image_path = filename  # Use full path from CSV
    img_array = load_and_preprocess_image(image_path)
    if img_array is not None:
        images.append(img_array)
        valid_targets.append(target)

# Convert to Numpy Arrays
images = np.array(images)
targets = np.array(valid_targets)

# Split Data
X_train, X_temp, y_train, y_temp = train_test_split(images, targets, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training images shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Validation images shape: {X_val.shape}, Validation labels shape: {y_val.shape}")
print(f"Testing images shape: {X_test.shape}, Testing labels shape: {y_test.shape}")

# --- 2. CNN Model ---
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="linear")  # Regression Output
])

# --- 3. Compile Model ---
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# --- 4. Train Model ---
EPOCHS = 30
BATCH_SIZE = 32

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# --- 5. Evaluate Model ---
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Mean Absolute Error: {mae:.4f}")

# --- 6. Visualization ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history.history["mae"], label="Training MAE")
plt.plot(history.history["val_mae"], label="Validation MAE")
plt.legend()
plt.title("Mean Absolute Error")

plt.show()


Error loading image: /content/drive/MyDrive/train_images/7005be54cab1.png - [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/7005be54cab1.png'
Error loading image: /content/drive/MyDrive/train_images/702de9dcde32.png - [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/702de9dcde32.png'
Error loading image: /content/drive/MyDrive/train_images/6fe67fd7f5d1.png - [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/6fe67fd7f5d1.png'
Error loading image: /content/drive/MyDrive/train_images/6fe67482bfae.png - [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/6fe67482bfae.png'
Error loading image: /content/drive/MyDrive/train_images/70d0392397de.png - [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/70d0392397de.png'
Error loading image: /content/drive/MyDrive/train_images/70d657f8f503.png - [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images/70d657f8f5

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt

# --- 1. Load CSV & Image Paths ---
CSV_PATH = "/content/drive/MyDrive/train_images/train.csv"  # Change to your CSV file path
IMAGE_SIZE = (224, 224)  # Standard input size for CNN

# Read the CSV file
df = pd.read_csv(CSV_PATH)

# Check if expected columns exist
if "Image_Path" not in df.columns or "Label" not in df.columns:
    raise KeyError("CSV file must contain 'Image_Path' and 'Label' columns!")

# Get image paths & labels
image_paths = df["Image_Path"].values
labels = df["Label"].values.astype(float)  # Convert labels to float for regression

# --- 2. Image Preprocessing Function ---
def load_and_preprocess_image(image_path):
    if not os.path.exists(image_path):  # Check if file exists
        print(f"❌ Missing file: {image_path}")
        return None
    img = Image.open(image_path).convert("RGB")
    img = img.resize(IMAGE_SIZE)
    return np.array(img) / 255.0  # Normalize pixel values

# --- 3. Load All Images ---
images = [load_and_preprocess_image(path) for path in image_paths]
images = np.array([img for img in images if img is not None])  # Remove None values

# Ensure labels match image count
labels = labels[: len(images)]

# --- 4. Train-Validation-Test Split ---
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"✅ Training images: {X_train.shape}, Labels: {y_train.shape}")
print(f"✅ Validation images: {X_val.shape}, Labels: {y_val.shape}")
print(f"✅ Testing images: {X_test.shape}, Labels: {y_test.shape}")

# --- 5. CNN Model ---
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="linear")  # Regression output
])

# --- 6. Compile the Model ---
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# --- 7. Train the Model ---
EPOCHS = 30
BATCH_SIZE = 32

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# --- 8. Evaluate the Model ---
loss, mae = model.evaluate(X_test, y_test)
print(f"✅ Test Loss: {loss:.4f}")
print(f"✅ Test Mean Absolute Error: {mae:.4f}")

# --- 9. Plot Training History ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Loss Over Epochs")

plt.subplot(1, 2, 2)
plt.plot(history.history["mae"], label="Training MAE")
plt.plot(history.history["val_mae"], label="Validation MAE")
plt.legend()
plt.title("Mean Absolute Error Over Epochs")

plt.show()


ValueError: could not convert string to float: 'class_1'

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import cv2
import glob

# --- 1. Load CSV File ---
csv_path = "/content/drive/MyDrive/train_images/train.csv"  # Update this with your actual CSV path
df = pd.read_csv(csv_path)

# Convert categorical labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df["Label"])  # Convert 'class_1', 'class_2' -> 0, 1, 2, ...

# --- 2. Load Images Efficiently ---
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)  # Read image
    if img is None:
        print(f"Error loading image: {image_path}")  # Debugging for missing files
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (224, 224))  # Resize
    img = img / 255.0  # Normalize
    return img

# Apply function to load images
image_paths = df["Image_Path"].values
images = [load_and_preprocess_image(img_path) for img_path in image_paths]
images = np.array([img for img in images if img is not None])  # Remove None values

# Ensure labels and images have matching sizes
labels = labels[:len(images)]

# --- 3. Split Data ---
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# --- 4. CNN Model for Classification ---
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation="softmax")  # Multi-class classification
])

# --- 5. Compile Model ---
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# --- 6. Train Model ---
EPOCHS = 10
BATCH_SIZE = 32

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# --- 7. Evaluate Model ---
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# --- 8. Save Model (Optional) ---
model.save("/content/drive/MyDrive/trained_model.h5")

# --- 9. Visualize Training History ---
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("Accuracy")

plt.show()


Error loading image: /content/drive/MyDrive/train_images/7005be54cab1.png
Error loading image: /content/drive/MyDrive/train_images/702de9dcde32.png
Error loading image: /content/drive/MyDrive/train_images/6fe67fd7f5d1.png
Error loading image: /content/drive/MyDrive/train_images/6fe67482bfae.png
Error loading image: /content/drive/MyDrive/train_images/70d0392397de.png
Error loading image: /content/drive/MyDrive/train_images/70d657f8f503.png
Error loading image: /content/drive/MyDrive/train_images/705f508d1e42.png
Error loading image: /content/drive/MyDrive/train_images/70ed3ec68b94.png
Error loading image: /content/drive/MyDrive/train_images/709784f7fcc2.png
Error loading image: /content/drive/MyDrive/train_images/70f5caf5f305.png
Error loading image: /content/drive/MyDrive/train_images/7102f29e052e.png
Error loading image: /content/drive/MyDrive/train_images/711d1480d2e3.png
Error loading image: /content/drive/MyDrive/train_images/710b05a96e0f.png
Error loading image: /content/drive/My

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [19]:
import os

image_dir = "/content/drive/MyDrive/train_images"  # Update if your path is different

# List first 10 files in directory
if os.path.exists(image_dir):
    print("Files in directory:", os.listdir(image_dir)[:10])
else:
    print(f"Directory {image_dir} does not exist!")


Files in directory: ['train_images', 'train.gsheet', 'images_train.gsheet', 'train.csv']


In [20]:
import os

image_dir = "/content/drive/MyDrive/train_images"

if os.path.exists(image_dir):
    print("Files in train_images directory:", os.listdir(image_dir)[:10])
else:
    print(f"Directory {image_dir} does not exist!")


Files in train_images directory: ['train_images', 'train.gsheet', 'images_train.gsheet', 'train.csv']


In [21]:
!ls -lh "/content/drive/MyDrive/train_images"


total 10K
-rw------- 1 root root  181 Mar 27 01:25 images_train.gsheet
-rw------- 1 root root 5.0K Mar 27 01:30 train.csv
-rw------- 1 root root  181 Mar 26 14:58 train.gsheet
drwx------ 2 root root 4.0K Mar 27 01:08 train_images


In [22]:
!ls -lh "/content/drive/MyDrive/train_images"


total 10K
-rw------- 1 root root  181 Mar 27 01:25 images_train.gsheet
-rw------- 1 root root 5.0K Mar 27 01:30 train.csv
-rw------- 1 root root  181 Mar 26 14:58 train.gsheet
drwx------ 2 root root 4.0K Mar 27 01:08 train_images


In [23]:
import glob

image_files = glob.glob("/content/drive/MyDrive/train_images/*", recursive=True)

print(f"Total images found: {len(image_files)}")
print(image_files[:5])  # Print first 5 image paths


Total images found: 4
['/content/drive/MyDrive/train_images/train_images', '/content/drive/MyDrive/train_images/train.gsheet', '/content/drive/MyDrive/train_images/images_train.gsheet', '/content/drive/MyDrive/train_images/train.csv']


In [25]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/train.zip"  # Change to your ZIP file path
extract_path = "/content/drive/MyDrive/train_images/"

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/train.zip'

In [27]:
!unzip "/content/drive/MyDrive/train.zip" -d "/content/drive/MyDrive/train_images"


unzip:  cannot find or open /content/drive/MyDrive/train.zip, /content/drive/MyDrive/train.zip.zip or /content/drive/MyDrive/train.zip.ZIP.
